# Examples

In [ ]:
# Import simulation packages
import numpy as np
import centpy

In [ ]:
# Setup up visualization packages
%matplotlib notebook
from matplotlib import rcParams
rcParams['figure.dpi']  = 60
import matplotlib.pyplot as plt
from matplotlib import animation

## 1D Examples 

### Burgers (SD3)

We solve the inviscid Burgers' equation 

\begin{equation}
\partial_t u + \partial_x \left( \frac{u^2}{2}\right) = 0,
\end{equation}

on the domain $(x,t)\in([0,2\pi]\times[0,5])$ with initial data

\begin{equation}
u(x,0) = \sin x + \frac{1}{2} \sin\left(\frac{x}{2}\right)\,. 
\end{equation}

and periodic boundary conditions. The solution is computed using 400 cells and CFL number 0.75.

In [ ]:
# Parameters

pars = centpy.Setup1d(x_init=0.0, x_final=2.0 * np.pi, t_final=10, dt_out=0.05, J=400, cfl=0.75)

In [ ]:
# Burgers equation

class Burgers(centpy.Equation1d):

    def initial_data(self):
        return np.sin(self.x) + 0.5 * np.sin(0.5 * self.x)

    def boundary_conditions(self, u):
        u[0] = u[-4]
        u[1] = u[-3]
        u[-2] = u[2]
        u[-1] = u[3]

    def flux_x(self, u):
        return 0.5 * u * u

    def spectral_radius_x(self, u):
        return np.abs(u)
    
eqn_burgers = Burgers(pars)

In [ ]:
# Solver
soln_burgers = centpy.Solver1d(eqn_burgers)

# Solution
soln_burgers.solve()

In [ ]:
# Animation 
j0 = centpy.j0

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure()
ax = plt.axes(xlim=(soln_burgers.x_init,soln_burgers.x_final), ylim=(np.min(soln_burgers.u_n), np.max(soln_burgers.u_n)*1.1))

line_u, = ax.plot([], [], linewidth=1, marker='o', markersize=2)

plt.xlabel('x')
plt.ylabel('u')
plt.legend(["burgers"], loc=3, frameon=False)
 
# initialization function: plot the background of each frame
def init():
    line_u.set_data([], [])

# animation function.  This is called sequentially
def animate(i):
    line_u.set_data(soln_burgers.x[j0], soln_burgers.u_n[i,j0])
    
# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init, frames=soln_burgers.Nt, interval=100, blit=False)
 
plt.show()

### Euler (SD2)

We solve the Euler equations in 1D 

\begin{equation} 
\partial_t 
\begin{bmatrix} \rho \\ \rho v \\ E \end{bmatrix} 
+ 
\partial_x 
\begin{bmatrix} \rho v \\ \rho v^2 +p \\ v (E+p) \end{bmatrix} 
= 0 
\end{equation}

with the equation of state

\begin{equation}
p = (\gamma-1) \left(E-\frac{1}{2} \rho v^2\right), \qquad \gamma=1.4
\end{equation}

on the domain $(x,t)\in([0,1]\times[0,0.1])$ with initial data for a *Sod shock tube*:

\begin{equation}
(\rho, v, p)_{t=0} = 
\begin{cases}
(1,0,1) & \text{if} & 0<x\leq0.5 \\
(0.125, 0, 0.1) & \text{if} & 0.5<x<1
\end{cases}
\end{equation}

and Dirichlet boundary data set by initial data on each boundary. The solution is computed using 400 cells and CFL number 0.75.

In [ ]:
# Parameters
pars_euler = centpy.Setup1d(x_init=0.0, x_final=1.0, t_final=0.3, dt_out=0.004, J=400, cfl=0.75, scheme = 'sd2')

In [ ]:
# Euler equation class and instance

class Euler(centpy.Equation1d):
    def __init__(self, setup, gamma):
        self.gamma = gamma
        super().__init__(setup)

    def initial_data(self):
        u = np.zeros((self.J + 4, 3))
        midpoint = int(self.J / 2) + 2

        left_v = [1, 0, 1.0 / (self.gamma - 1.0)]
        right_v = [0.125, 0.0, 0.1 / (self.gamma - 1.0)]

        # Left side
        u[:midpoint, :] = left_v
        # Right side
        u[midpoint:, :] = right_v

        return u

    def boundary_conditions(self, u):
        left_v = [1, 0, 1.0 / (self.gamma - 1.0)]
        right_v = [0.125, 0.0, 0.1 / (self.gamma - 1.0)]
        # Left side
        u[0] = left_v
        u[1] = left_v
        # Right side
        u[-1] = right_v
        u[-2] = right_v

    def flux_x(self, u):
        f = np.zeros_like(u)
        rho = u[:, 0]
        u_x = u[:, 1] / rho
        E = u[:, 2]
        p = (self.gamma - 1.0) * (E - 0.5 * rho * u_x ** 2)

        f[:, 0] = rho * u_x
        f[:, 1] = rho * u_x ** 2 + p
        f[:, 2] = u_x * (E + p)

        return f

    def spectral_radius_x(self, u):
        rho = u[:, 0]
        u_x = u[:, 1] / rho
        p = (self.gamma - 1.0) * (u[:, 2] - 0.5 * rho * u_x ** 2)
        return np.abs(u_x) + np.sqrt(self.gamma * p / rho)
    
# Equation instance
eqn_euler = Euler(pars_euler, gamma=1.4)

In [ ]:
# Solver instance
soln_euler = centpy.Solver1d(eqn_euler)

# Solution
soln_euler.solve()

In [ ]:
# Animation 
u_n = soln_euler.u_n.copy()
x   = soln_euler.x.copy()
gamma = soln_euler.gamma 
Nt = soln_euler.Nt 

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure(figsize=(10,8))
ax1=fig.add_subplot(1,2,1)
ax2=fig.add_subplot(2,2,2)
ax3=fig.add_subplot(2,2,4)
#ax = plt.axes(xlim=(x_init,x_final), ylim=(np.min(u_n), np.max(u_n)*1.1))

line_u1, = ax1.plot([], [], linewidth=1, marker='o', markersize=2)
line_u2, = ax2.plot([], [], linewidth=1, marker='o', markersize=2)
line_u3, = ax3.plot([], [], linewidth=1, marker='o', markersize=2)

ax1.set_xlabel('x')
ax3.set_xlabel('x')
ax1.set_ylabel(r'$\rho$')
ax2.set_ylabel(r'$v_x$')
ax3.set_ylabel(r'$p$')
ax1.set_ylim(0.1, 1.05)
ax2.set_ylim(0, 1)
ax3.set_ylim(0.05, 1.1)
#plt.legend(["u0","u1","u2"], loc=3, frameon=False)

plt.subplots_adjust(bottom=0.1, right=0.8, top=0.9)

# initialization function: plot the background of each frame
def init():
    line_u1.set_data([], [])
    line_u2.set_data([], [])
    line_u3.set_data([], [])

# animation function.  This is called sequentially
def animate(i):
    rho=u_n[i,j0,0]
    u_x = u_n[i,j0,1]/u_n[i,j0,0]
    p = (gamma-1.)*(u_n[i,j0,2]-0.5*rho*u_x**2)
    line_u1.set_data(x[j0], rho)
    line_u2.set_data(x[j0], u_x)
    line_u3.set_data(x[j0], p)
    
# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init, frames=Nt, interval=100, blit=False)
 
plt.show()

### MHD (FD2)

We solve the equations of ideal magnetohydrodynamics in 1D 

\begin{equation} 
\partial_t 
\begin{bmatrix} 
\rho \\ \rho v_x \\ \rho v_y \\ \rho v_z \\ B_y \\ B_z \\ E 
\end{bmatrix} 
+ 
\partial_x 
\begin{bmatrix} 
\rho v_x \\ \rho v_x^2 + p^* - B_x^2 \\ \rho v_x v_y - B_x B_y \\
\rho v_x v_z - B_x B_z \\ B_y v_x - B_x v_y \\ B_z v_x - B_x v_z \\
(E+p^*) v_x - B_x (B_x v_x + B_y v_y + B_z v_Z) 
\end{bmatrix} 
= 0 
\end{equation}

where the total pressure is given by 

\begin{equation}
p^* = p + \frac{1}{2} (B_x^2 + B_y^2 + B_z^2)
\end{equation}

with the equation of state

\begin{equation}
p = (\gamma-1) \left(E-\frac{1}{2} \rho (v_x^2+v_y^2+v_z^2) - 
\frac{1}{2}(B_x^2 + B_y^2 + B_z^2)\right), \qquad \gamma=2.0
\end{equation}

The solution is computed on the domain $(x,t)\in([-1,1]\times[0,0.2])$ with initial data for a *Brio-Wu shock tube*:

\begin{equation}
(\rho, v_x, v_y, v_z, B_y, B_z, p)_{t=0} = 
\begin{cases}
(1,0,0,0,1,0,1) & \text{if} & -1<x\leq 0 \\
(0.125, 0, 0, 0, -1, 0, 0.1) & \text{if} & \ \ 0<x<1
\end{cases}
\end{equation}

and Dirichlet boundary data set by initial data on each boundary. The solution is computed using 400 cells and CFL number 0.475.

In [ ]:
# Parameters
pars_mhd = centpy.Setup1d(x_init=-1.0, x_final=1.0, t_final=0.2, dt_out=0.002, J=400, cfl=0.475, scheme = 'fd2')

In [ ]:
# MHD equation

class MHD(centpy.Equation1d):
    def __init__(self, setup, B1):
        self.B1 = B1
        super().__init__(setup)

    def pressure(self,u):
        return u[:,6]-0.5*((u[:,1]**2+u[:,2]**2+u[:,3]**2)/u[:,0])-0.5*(self.B1**2+u[:,4]**2+u[:,5]**2)

    def initial_data(self):
        u=np.zeros((self.J+4,7))
        midpoint = int(self.J/2)+2

        # Left side
        u[:midpoint,0] = 1.
        u[:midpoint,1] = 0.
        u[:midpoint,2] = 0.
        u[:midpoint,3] = 0.
        u[:midpoint,4] = 1.
        u[:midpoint,5] = 0.
        u[:midpoint,6] = 1.+25./32.

        # Right side
        u[midpoint:,0] = 0.125
        u[midpoint:,1] = 0.
        u[midpoint:,2] = 0.
        u[midpoint:,3] = 0.
        u[midpoint:,4] = -1.
        u[midpoint:,5] = 0.
        u[midpoint:,6] = 0.1+25./32.    

        return u

    def boundary_conditions(self,u):

        left_v = [1., 0.,0.,0.,1.,0.,1.+25./32.]
        right_v = [0.125,0.,0.,0.,-1.,0.,.1+25./32]

        if(centpy.odd):
            u[0] = left_v
            u[-1]= right_v
            u[-2]= right_v
        else:
            u[0] = left_v
            u[1] = left_v
            u[-1]= right_v

#     def boundary_conditions(self, u): # if using a non-staggered grid

#         left_v = [1., 0.,0.,0.,1.,0.,1.+25./32.]
#         right_v = [0.125,0.,0.,0.,-1.,0.,.1+25./32]

#         u[0] = left_v
#         u[1] = left_v
#         u[-1]= right_v
#         u[-2]= right_v

#         return u    
    
    def flux_x(self,u):
        f = np.zeros_like(u)
        
        B1 = self.B1
        
        p_star = self.pressure(u)+0.5*(B1**2+u[:,4]**2+u[:,5]**2)

        f[:,0] = u[:,1]
        f[:,1] = u[:,1]**2/u[:,0]+p_star
        f[:,2] = u[:,1]*u[:,2]/u[:,0]-B1*u[:,4]
        f[:,3] = u[:,1]*u[:,3]/u[:,0]-B1*u[:,5]
        f[:,4] = u[:,1]*u[:,4]/u[:,0]-B1*u[:,2]/u[:,0]
        f[:,5] = u[:,1]*u[:,5]/u[:,0]-B1*u[:,3]/u[:,0]
        f[:,6] = (u[:,6]+p_star)*(u[:,1]/u[:,0]) -B1*(B1*u[:,1]+u[:,2]*u[:,4]+u[:,3]*u[:,5])/u[:,0]

        return f

    def spectral_radius_x(self, u):
        rho=u[:,0]  
        u_x = u[:,1]/rho
        p = self.pressure(u)
        A = 2.0*p/rho
        B = (self.B1**2 + u[:,4]**2 + u[:,5]**2)/rho
        cf = np.sqrt(0.5*(A+B+np.sqrt((A+B)**2-4.*A*self.B1**2/rho)))
        return np.abs(u_x) + cf
    
# Equation
eqn_mhd = MHD(pars_mhd, B1=0.75)

In [ ]:
# Solver instance
soln_mhd = centpy.Solver1d(eqn_mhd)

# Solution
soln_mhd.solve()

In [ ]:
# Animation 
 
u_n = soln_mhd.u_n.copy()
x   = soln_mhd.x.copy()
Nt  = soln_mhd.Nt 
pressure = eqn_mhd.pressure
# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure(figsize=(12,6))
ax1=fig.add_subplot(1,3,1)
ax2=fig.add_subplot(2,3,2)
ax3=fig.add_subplot(2,3,3)
ax4=fig.add_subplot(2,3,5)
ax5=fig.add_subplot(2,3,6)
#ax = plt.axes(xlim=(x_init,x_final), ylim=(np.min(u_n), np.max(u_n)*1.1))

line_u1, = ax1.plot([], [], linewidth=1, marker='o', markersize=2)
line_u2, = ax2.plot([], [], linewidth=1, marker='o', markersize=2)
line_u3, = ax3.plot([], [], linewidth=1, marker='o', markersize=2)
line_u4, = ax4.plot([], [], linewidth=1, marker='o', markersize=2)
line_u5, = ax5.plot([], [], linewidth=1, marker='o', markersize=2)

ax1.set_xlabel('x')
ax4.set_xlabel('x')
ax5.set_xlabel('x')

ax2.set_xticks([])
ax3.set_xticks([])
ax2.set_yticks([])
ax3.set_yticks([])
ax4.set_yticks([])
ax5.set_yticks([])

ax1.set_ylabel(r'$\rho$', fontsize=12)
ax2.set_ylabel(r'$v_x$', fontsize=12)
ax3.set_ylabel(r'$v_y$', fontsize=12)
ax4.set_ylabel(r'$B_y$', fontsize=12)
ax5.set_ylabel(r'$p$', fontsize=12)

# Primitive variables
rho=u_n[:,j0,0]
v_x = u_n[:,j0,1]/u_n[:,j0,0]
v_y = u_n[:,j0,2]/u_n[:,j0,0]
B_y = u_n[:,j0,4]

ax1.set_xlim(-1, 1)
ax2.set_xlim(-1, 1)
ax3.set_xlim(-1, 1)
ax4.set_xlim(-1, 1)
ax5.set_xlim(-1, 1)
ax1.set_ylim(np.min(rho), np.max(rho))
ax2.set_ylim(np.min(v_x), np.max(v_x))
ax3.set_ylim(np.min(v_y), np.max(v_y))
ax4.set_ylim(np.min(B_y), np.max(B_y))
ax5.set_ylim(0.05, 1.)

#plt.subplots_adjust(bottom=0.1, right=0.8, top=0.9)

# initialization function: plot the background of each frame
def init():
    line_u1.set_data([], [])
    line_u2.set_data([], [])
    line_u3.set_data([], [])
    line_u4.set_data([], [])
    line_u5.set_data([], [])

# animation function.  This is called sequentially
def animate(i):
    p = pressure(u_n[i,j0,:])
    line_u1.set_data(x[j0], rho[i])
    line_u2.set_data(x[j0], v_x[i])
    line_u3.set_data(x[j0], v_y[i])
    line_u4.set_data(x[j0], B_y[i])
    line_u5.set_data(x[j0], p)
    
# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init, frames=Nt, interval=100, blit=False)
 
plt.show()

## 2D Examples

### Scalar (SD3)

We solve the nonlinear scalar conservation law 

\begin{equation}
\partial_t u + \partial_x \sin u + \frac{1}{3} \partial_y u^3= 0,
\end{equation}

on the domain $(x,y,t)\in([0,2\pi]\times[0,2\pi]\times[0,6])$ with initial data

\begin{equation}
u(x,y,0) = \sin \left(x+\frac{1}{2}\right) \cos(2x+y)
\end{equation}

and periodic boundary conditions. The solution is computed using a 144 $\times$ 144 mesh and CFL number 0.9.

In [ ]:
# Parameters
pars_scalar = centpy.Setup2d(
    x_init = 0.0, x_final = 2.*np.pi,
    y_init = 0.0, y_final = 2.*np.pi,
    J = 144, K = 144,
    t_final = 6.0, dt_out = 0.1, cfl = 0.9, scheme='sd3')

In [ ]:
# Equation

class Scalar(centpy.Equation2d):

    def flux_x(self, u):
        return np.sin(u)

    def flux_y(self, u):
        return 1./3.*u**3

    def initial_data(self): 
        return np.sin(self.xx.T+0.5)*np.cos(2*self.xx.T+self.yy.T)
    
    def boundary_conditions(self,u):
        # x-boundary
        u[0] = u[-4]
        u[1] = u[-3]
        u[-2] = u[2]
        u[-1] = u[3]
        # y-boundary
        u[:,0] = u[:,-4]
        u[:,1] = u[:,-3]
        u[:,-2] = u[:,2]
        u[:,-1] = u[:,3]    

    def spectral_radius_x(self,u):
        return np.abs(np.cos(u))

    def spectral_radius_y(self,u):
        return u**2
    
# Instance
eqn_scalar = Scalar(pars_scalar)

In [ ]:
# Solution
soln_scalar = centpy.Solver2d(eqn_scalar)
soln_scalar.solve()

In [ ]:
# Animation
x_init = soln_scalar.x_init
x_final = soln_scalar.x_final
y_init = soln_scalar.y_init
y_final = soln_scalar.y_final
x = soln_scalar.x
y = soln_scalar.y
u_n = soln_scalar.u_n
Nt = soln_scalar.Nt
j0 = centpy.j0


fig = plt.figure()
ax = plt.axes(xlim=(x_init,x_final), ylim=(y_init, y_final))

ax.set_title("Nonlinear scalar")
ax.set_xlabel("x")
ax.set_ylabel("y")

contours=ax.contour(x[j0], y[j0], u_n[0,j0,j0], 8, colors='black') 
img=ax.imshow(u_n[0,j0,j0], extent=[0, 6.3, 0, 6.3], origin='lower',
           cmap='ocean', alpha=0.5)
#cl=ax.clabel(contours, inline=True, fontsize=8)

fig.colorbar(img)
plt.axis(aspect='image')
def animate(i):
    ax.collections = []
    ax.contour(x[j0], y[j0], u_n[i,j0,j0], 8, colors='black') 
#    cl=ax.clabel(contours, inline=True, fontsize=8)
    img.set_array(u_n[i,j0,j0])
    img.autoscale()
    
# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, frames=Nt, interval=100, blit=False)
 
plt.show()

### Euler (FD2)

We solve the Euler equations in 2D 

\begin{equation} 
\partial_t 
\begin{bmatrix} \rho \\ \rho u_x \\ rho u_y \\ E \end{bmatrix} 
+ 
\partial_x 
\begin{bmatrix} \rho u_x \\ \rho u_x^2 + p \\  \rho u_x u_y \\ (E+p) u_x \end{bmatrix} 
+ 
\partial_y 
\begin{bmatrix} \rho u_y \\ \rho u_y u_x \\  \rho u_y^2 +p \\ (E+p) u_y \end{bmatrix} 
= 0 
\end{equation}

with the equation of state

\begin{equation}
p = (\gamma-1) \left(E-\frac{1}{2} \rho (u_x^2 - u_y^2) \right), \qquad \gamma=1.4
\end{equation}

on the domain $(x,y,t)\in([0,1]\times[0,1]\times[0,0.1])$ with initial data for a *2D Riemann problem*:

\begin{equation}
(\rho, v, p)_{t=0} = 
\begin{cases}
(1,0,1) & \text{if} & 0<x\leq0.5 \\
(0.125, 0, 0.1) & \text{if} & 0.5<x<1
\end{cases}
\end{equation}

and Dirichlet boundary data set by initial data on each boundary. The solution is computed using a 200 $\times$ 200 mesh and CFL number 0.75.

In [ ]:
# Parameters
pars_euler = centpy.schemes2d.Setup2d(
    x_init = 0.0, x_final = 1.0,
    y_init = 0.0, y_final = 1.0,
    J = 200, K = 200,
    t_final = 0.4, dt_out = 0.005, cfl = 0.475, scheme='fd2')

In [ ]:
# Helper functions for the equation
def pressure(u, gamma):
    return (gamma-1.0)*(u[:,:,3]-0.5*(u[:,:,1]**2+u[:,:,2]**2)/u[:,:,0])

def euler_data(gamma):

    p_one=1.5
    p_two=0.3
    p_three=0.029
    p_four=0.3
      
    upper_right,upper_left,lower_right,lower_left = np.ones((4,4))
    
    upper_right[0] = 1.5
    upper_right[1] = 0.0
    upper_right[2] = 0.0
    upper_right[3] = p_one/(gamma - 1.0) + 0.5*(upper_right[1]**2 + upper_right[2]**2)/upper_right[0]

    upper_left[0] = 0.5323
    upper_left[1] = 1.206*upper_left[0]
    upper_left[2] = 0.0
    upper_left[3] = p_two/(gamma - 1.0) + 0.5*(upper_left[1]**2 + upper_left[2]**2)/upper_left[0]

    lower_right[0] = 0.5323
    lower_right[1] = 0.0
    lower_right[2] = 1.206*lower_right[0]
    lower_right[3] = p_four/(gamma - 1.0) + 0.5*(lower_right[1]**2 + lower_right[2]**2)/lower_right[0]

    lower_left[0] = 0.138
    lower_left[1] = 1.206*lower_left[0]
    lower_left[2] = 1.206*lower_left[0]
    lower_left[3] = p_three/(gamma - 1.0) + 0.5*(lower_left[1]**2 + lower_left[2]**2)/lower_left[0]

    return upper_right,upper_left,lower_right,lower_left

In [ ]:
# Equation

class Euler(centpy.Equation2d):

    j0 = centpy.j0
    
    def __init__(self, setup, gamma):
        self.gamma = gamma
        super().__init__(setup)

    def initial_data(self): 
        u=np.empty((self.J+4,self.K+4,4))
        midJ = int(self.J/2)+2
        midK = int(self.K/2)+2

        one_matrix = np.ones(u[midJ:,midK:].shape)
        upper_right,upper_left,lower_right,lower_left = euler_data(self.gamma)

        u[midJ:,midK:] = upper_right*one_matrix
        u[:midJ,midK:] = upper_left*one_matrix
        u[midJ:,:midK] = lower_right*one_matrix
        u[:midJ,:midK] = lower_left*one_matrix
        return u

    def boundary_conditions(self, u):

        upper_right,upper_left,lower_right,lower_left = euler_data(self.gamma)
        
        if(centpy.odd):
            j=slice(1,-2)
            u[j,0] = u[j,1]
            u[j,-2]= u[j,-3]
            u[j,-1]= u[j,-3]

            u[0,j] = u[1,j]
            u[-2,j]= u[-3,j]
            u[-1,j]= u[-3,j]

             # one
            u[-2,-2] = upper_right 
            u[-1,-2] = upper_right
            u[-2,-1] = upper_right
            u[-1,-1] = upper_right

            # two
            u[0,-2] = upper_left
            u[0,-1] = upper_left 

            # three
            u[0,0] = lower_left
            u[0,1] = lower_left
            u[1,0] = lower_left
            u[1,1] = lower_left    

            # four
            u[-2,0] = lower_right
            u[-1,0] = lower_right
            u[-2,1] = lower_right
            u[-1,1] = lower_right

        else:

            j=slice(2,-1)
            u[j,0] = u[j,2]
            u[j,1] = u[j,2]
            u[j,-1]= u[j,-2]

            u[0,j] = u[2,j]
            u[1,j] = u[2,j]
            u[-1,j]= u[-2,j]

            # one
            u[-1,-2] = upper_right 
            u[-1,-1] = upper_right

            # two
            u[0,-2] = upper_left
            u[0,-1] = upper_left 
            u[1,-2] = upper_left 
            u[1,-1] = upper_left 

            # three
            u[0,0] = lower_left
            u[0,1] = lower_left
            u[1,0] = lower_left
            u[1,1] = lower_left

            # four
            u[-1,0] = lower_right 
            u[-1,1] = lower_right
    
    def flux_x(self,u):
        f = np.empty_like(u)

        p=pressure(u, self.gamma)

        f[:,:,0]=u[:,:,1]
        f[:,:,1]=u[:,:,1]**2/u[:,:,0] + p
        f[:,:,2]=u[:,:,1]*u[:,:,2]/u[:,:,0]
        f[:,:,3]=(u[:,:,3]+p)*u[:,:,1]/u[:,:,0]  

        return f

    def flux_y(self, u):
        g = np.empty_like(u)

        p=pressure(u, self.gamma)

        g[:,:,0]=u[:,:,2]
        g[:,:,1]=u[:,:,1]*u[:,:,2]/u[:,:,0]
        g[:,:,2]=u[:,:,2]**2/u[:,:,0] + p
        g[:,:,3]=(u[:,:,3]+p)*u[:,:,2]/u[:,:,0]  

        return g                   
    
    def spectral_radius_x(self, u):
        rho=u[j0,j0,0]
        vx=u[j0,j0,1]/rho
        vy=u[j0,j0,2]/rho
        p=(self.gamma-1.0)*(u[j0,j0,3]-.5*rho*(vx**2 + vy**2))
        c=np.sqrt(self.gamma*p/rho)
        return np.abs(vx)+c

    def spectral_radius_y(self,u):
        rho=u[j0,j0,0]
        vx=u[j0,j0,1]/rho
        vy=u[j0,j0,2]/rho
        p=(self.gamma-1.0)*(u[j0,j0,3]-.5*rho*(vx**2 + vy**2))
        c=np.sqrt(self.gamma*p/rho)
        return np.abs(vy)+c
    
# Equation instance
eqn_euler = Euler(pars_euler, gamma=1.4)

In [ ]:
# Solution
soln_euler = centpy.Solver2d(eqn_euler)
soln_euler.solve()

In [ ]:
# Animation
x_init = soln_euler.x_init
x_final = soln_euler.x_final
y_init = soln_euler.y_init
y_final = soln_euler.y_final
x = soln_euler.x
y = soln_euler.y
u_n = soln_euler.u_n
Nt = soln_euler.Nt

fig = plt.figure()
ax = plt.axes(xlim=(x_init,x_final), ylim=(y_init, y_final))
#levels=np.arange(-1, 1, 0.1) 

ax.contour(x[1:-1], y[1:-1], u_n[0,1:-1,1:-1,0]) 
def animate(i):
    ax.collections = []
    ax.contour(x[1:-1], y[1:-1], u_n[i,1:-1,1:-1,0])
    
# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, frames=Nt, interval=100, blit=False)
 
plt.show()

### MHD (SD2)

We solve the equations of ideal magnetohydrodynamics in 1D 

\begin{equation} 
\partial_t 
\begin{bmatrix} 
\rho \\ \rho v_x \\ \rho v_y \\ \rho v_z \\ B_x \\ B_y \\ B_z \\ E 
\end{bmatrix} 
+ 
\partial_x 
\begin{bmatrix} 
\rho v_x \\ \rho v_x^2 + p^* - B_x^2 \\ \rho v_x v_y - B_x B_y \\
\rho v_x v_z - B_x B_z \\ 0 \\ B_y v_x - B_x v_y \\ B_z v_x - B_x v_z \\
(E+p^*) v_x - B_x (B_x v_x + B_y v_y + B_z v_Z) 
\end{bmatrix} 
+ 
\partial_y
\begin{bmatrix} 
\rho v_y \\ \rho v_y v_x - B_y B_x \\ \rho v_y^2 + p^* - B_y^2  \\
\rho v_y v_z - B_y B_z \\ B_x v_y - B_y v_x \\ 0 \\ B_z v_y - B_y v_z \\
(E+p^*) v_y - B_y (B_x v_x + B_y v_y + B_z v_Z) 
\end{bmatrix} 
= 0 
\end{equation}

where the total pressure is given by 

\begin{equation}
p^* = p + \frac{1}{2} (B_x^2 + B_y^2 + B_z^2)
\end{equation}

with the equation of state

\begin{equation}
p = (\gamma-1) \left(E-\frac{1}{2} \rho (v_x^2+v_y^2+v_z^2) - 
\frac{1}{2}(B_x^2 + B_y^2 + B_z^2)\right), \qquad \gamma=5/3
\end{equation}

The solution is computed on the domain $(x,y,t)\in([0,2\pi]\times[0,2\pi]\times[0,3])$ with initial data for an *Orszag-Tang vortex system*:

\begin{equation}
(\rho, v_x, v_y, v_z, B_x, B_y, B_z, p)_{t=0} = 
(\gamma^2, -\sin y, \sin x, 0, -\sin y, \sin (2x), 0, \gamma)
\end{equation}

and Dirichlet boundary data set by initial data on each boundary. The solution is computed using a 288$\times$288 mesh and CFL number 0.75.

In [ ]:
# Parameters
pars_mhd = centpy.Setup2d(
    x_init = 0.0, x_final = 2.*np.pi,
    y_init = 0.0, y_final = 2.*np.pi,
    J = 144, K = 144,
    t_final = 3.0, dt_out = 0.05, cfl = 0.75, scheme='sd2')

In [ ]:
# Equation

# Helper functions for the equation

def pressure(u, gamma):
    return (gamma-1.0)*(u[:,:,7]-0.5*(u[:,:,1]**2+u[:,:,2]**2+u[:,:,3]**2)/u[:,:,0]\
                     -0.5*(u[:,:,4]**2+u[:,:,5]**2+u[:,:,6]**2))

def pressure_star(u, gamma):
    return pressure(u, gamma) + 0.5*(u[:,:,4]**2+u[:,:,5]**2+u[:,:,6]**2) 

# Equation class
class MHD(centpy.Equation2d):
    def __init__(self, setup, gamma):
        self.gamma = gamma
        super().__init__(setup)

    def initial_data(self): 
        u=np.zeros((self.J+4,self.K+4,8))
        x=self.xx.T; y=self.yy.T
        gamma = self.gamma
        dx = self.dx; dy = self.dy
        
        u[:,:,0] = gamma**2
        u[:,:,1] = u[:,:,0]/dy*(np.cos(y+0.5*dy)-np.cos(y-0.5*dy))
        u[:,:,2] = 0.
        u[:,:,3] = -u[:,:,0]/dx*(np.cos(x+0.5*dx)-np.cos(x-0.5*dx))
        u[:,:,4] = 1./dy*(np.cos(y+0.5*dy)-np.cos(y-0.5*dy))
        u[:,:,5] = 0.
        u[:,:,6] = -0.5/dx*(np.cos(2.*(x+0.5*dx))-np.cos(2.*(x-0.5*dx)))

        I1 =-0.125/dy*(u[:,:,0]+1.)*(np.sin(2.*(y+0.5*dy))-np.sin(2.*(y-0.5*dy)))
        I2 =-0.125/dx*u[:,:,0]*(np.sin(2.*(x+0.5*dx))-np.sin(2.*(x-0.5*dx)))
        I3 =-0.0625/dx*(np.sin(4.*(x+0.5*dx))-np.sin(4.*(x-0.5*dx)))
        u[:,:,7] = 3.+0.5*u[:,:,0]+I1+I2+I3    

        return u

    def boundary_conditions(self, u): # periodic
        # x-boundary
        u[0] = u[-4]
        u[1] = u[-3]
        u[-2] = u[2]
        u[-1] = u[3]
        # y-boundary
        u[:,0] = u[:,-4]
        u[:,1] = u[:,-3]
        u[:,-2] = u[:,2]
        u[:,-1] = u[:,3]    

    def flux_x(self, u):
        f = np.empty_like(u)

        p_star=pressure_star(u, self.gamma)

        f[:,:,0]=u[:,:,1]
        f[:,:,1]=u[:,:,1]**2/u[:,:,0] + p_star - u[:,:,4]**2
        f[:,:,2]=u[:,:,1]*u[:,:,2]/u[:,:,0] - u[:,:,4]*u[:,:,5]
        f[:,:,3]=u[:,:,1]*u[:,:,3]/u[:,:,0] - u[:,:,4]*u[:,:,6]
        f[:,:,4]=0.
        f[:,:,5]=u[:,:,1]*u[:,:,5]/u[:,:,0] - u[:,:,4]*u[:,:,2]/u[:,:,0]
        f[:,:,6]=u[:,:,1]*u[:,:,6]/u[:,:,0] - u[:,:,4]*u[:,:,3]/u[:,:,0]
        f[:,:,7]=(u[:,:,7]+p_star)*u[:,:,1]/u[:,:,0] - u[:,:,4]*(u[:,:,4]*u[:,:,1]/u[:,:,0]+u[:,:,5]*u[:,:,2]/u[:,:,0]+\
                                                                 u[:,:,6]*u[:,:,3]/u[:,:,0])

        return f

    def flux_y(self, u):
        g = np.empty_like(u)

        p_star=pressure_star(u, self.gamma)

        g[:,:,0]=u[:,:,3]
        g[:,:,1]=u[:,:,3]*u[:,:,1]/u[:,:,0] - u[:,:,4]*u[:,:,6]
        g[:,:,2]=u[:,:,3]*u[:,:,2]/u[:,:,0] - u[:,:,5]*u[:,:,6]
        g[:,:,3]=u[:,:,3]**2/u[:,:,0] + p_star - u[:,:,6]**2

        g[:,:,4]=u[:,:,3]*u[:,:,4]/u[:,:,0] - u[:,:,6]*u[:,:,1]/u[:,:,0]
        g[:,:,5]=u[:,:,3]*u[:,:,5]/u[:,:,0] - u[:,:,6]*u[:,:,2]/u[:,:,0]
        g[:,:,6]=0.
        g[:,:,7]=(u[:,:,7]+p_star)*u[:,:,3]/u[:,:,0] - u[:,:,6]*(u[:,:,4]*u[:,:,1]/u[:,:,0]+u[:,:,5]*u[:,:,2]/u[:,:,0]+\
                                                                 u[:,:,6]*u[:,:,3]/u[:,:,0])    

        return g                   
        
    def spectral_radius_x(self, u):   
        rho=u[:,:,0]
        vx=u[:,:,1]/rho
        vy=u[:,:,2]/rho
        vz=u[:,:,3]/rho
        p=(self.gamma-1.0)*(u[:,:,7]-.5*rho*(vx**2 + vy**2 + vz**2) - 0.5*(u[:,:,4]**2+u[:,:,5]**2+u[:,:,6]**2))
        A=self.gamma*p/rho
        B=(u[:,:,4]**2+u[:,:,5]**2+u[:,:,6]**2)/rho
        cfx=np.sqrt(0.5*(A+B+np.sqrt((A+B)**2-4*A*u[:,:,4]**2/rho)))
        cfy=np.sqrt(0.5*(A+B+np.sqrt((A+B)**2-4*A*u[:,:,6]**2/rho)))

        return np.abs(vx)+cfx

    def spectral_radius_y(self, u):
        rho=u[:,:,0]
        vx=u[:,:,1]/rho
        vy=u[:,:,2]/rho
        vz=u[:,:,3]/rho
        p=(self.gamma-1.0)*(u[:,:,7]-.5*rho*(vx**2 + vy**2 + vz**2) - 0.5*(u[:,:,4]**2+u[:,:,5]**2+u[:,:,6]**2))
        A=self.gamma*p/rho
        B=(u[:,:,4]**2+u[:,:,5]**2+u[:,:,6]**2)/rho
        cfx=np.sqrt(0.5*(A+B+np.sqrt((A+B)**2-4*A*u[:,:,4]**2/rho)))
        cfy=np.sqrt(0.5*(A+B+np.sqrt((A+B)**2-4*A*u[:,:,6]**2/rho)))

        return np.abs(vy)+cfy
    
# Equation instance
eqn_mhd = MHD(pars_mhd, gamma=5./3)

In [ ]:
# Solution
soln_mhd = centpy.Solver2d(eqn_mhd)
soln_mhd.solve()

In [ ]:
# Animation

x_init = soln_mhd.x_init
x_final = soln_mhd.x_final
y_init = soln_mhd.y_init
y_final = soln_mhd.y_final
x = soln_mhd.x
y = soln_mhd.y
u_n = soln_mhd.u_n
Nt = soln_mhd.Nt
j = centpy.j0

fig = plt.figure()
ax = plt.axes(xlim=(x_init,x_final), ylim=(y_init, y_final))

ax.set_title("MHD Density")
ax.set_xlabel("x")
ax.set_ylabel("y")

#contours=ax.contour(x[j], y[j], u_n[0,j,j,0], 6, colors='black') 
img=ax.imshow(u_n[0,j,j,0], extent=[0, 6.3, 0, 6.3], origin='lower',
           cmap='inferno', alpha=0.5)
#cl=ax.clabel(contours, inline=True, fontsize=8)

fig.colorbar(img)
plt.axis(aspect='image')
def animate(i):
    ax.collections = []
    ax.contour(x[j], y[j], u_n[i,j,j,0], 20, colors='black',linewidths=0.1) 
    img.set_array(u_n[i,j,j,0])
    img.autoscale()
    
# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, frames=Nt, interval=50, blit=False)
 
plt.show()

In [ ]:
# Wireframe
xx = soln_mhd.xx
yy = soln_mhd.yy

from mpl_toolkits.mplot3d import axes3d

fig = plt.figure()
ax = axes3d.Axes3D(fig)

wframe = ax.plot_wireframe(xx, yy, u_n[0,:,:,0], rstride=2, cstride=2)
#ax.set_zlim(-1,1)

def update(i, ax, fig):
    ax.cla()
    wframe = ax.plot_wireframe(xx,yy,u_n[i,:,:,0], rstride=2, cstride=2)
    #ax.set_zlim(-1,1)
    return wframe,

ani = animation.FuncAnimation(fig, update, 
        frames=Nt, 
        fargs=(ax, fig), interval=100)
plt.show()